*   * *

**CHATLOG:**

*   **Inicio de la conversación:**
    *   Se proporcionó una guía para el desarrollo del pipeline de ingesta con CrewAI.
    *   Se generó un plan de acción detallado para implementar el pipeline.
    *   Se configuraron y verificaron los secretos de Supabase (`SUPABASE_URL` y `SUPABASE_API`).
    *   Se estableció la conexión a Supabase con éxito.
    *   Se estableció la conexión a Pinecone con éxito.
    *   Se desplego el modelo "sentence-transformers/all-MiniLM-L6-v2" para pasar texto a embeddings.
    *   Se crearon archivos en el directorio del cuaderno para proporcionar contexto a Gemini en Colab.
    *   Se proporcionaron los archivos "Fuentes\_Inverbot\_Pipeline.md" y "Estructura\_Datos\_Inverbot (1).md" detallando las fuentes de datos y la estructura de las bases de datos (Supabase y Pinecone).
    *   Se analizó el problema de la ingesta de datos considerando la diversidad de fuentes y la doble salida a bases de datos.
    *   Se presentaron opciones de diseño/desarrollo para el pipeline de CrewAI.
    *   Se seleccionó la **Opción 2: Diseño por Etapas del Pipeline (Extracción -> Procesamiento -> Carga)**.
    *   Se aclararon detalles sobre las librerías a utilizar, el rol de los modelos `sentence-transformers/all-MiniLM-L6-v0.3` y `mistralai/Mistral-7B-Instruct-v0.3`, el manejo de datos nuevos/actualizados para evitar duplicados y la modularidad de las fuentes.
    *   Se generó un plan detallado para la implementación de la Opción 2, incluyendo la configuración, el diseño de agentes/tareas, la implementación de cada etapa (Extracción, Procesamiento Estructurado, Procesamiento Vectorial, Carga), el desarrollo de tests y el refinamiento/optimización.
    *   Se inició la implementación del paso 1 del plan: Configuración Inicial y Despliegue de Modelos, enfocándose en el despliegue del modelo `mistralai/Mistral-7B-Instruct-v0.3` como un modelo self-hosted desde Hugging Face, y se generó el código para su instalación y carga.
    *   Se confirmó el despliegue exitoso del modelo `mistralai/Mistral-7B-Instruct-v0.3` en el entorno del notebook con GPU A100.
    *   Se comenzó con el paso 2 del plan: Diseño de Agentes y Tareas de CrewAI, definiendo los roles generales para las etapas de Extracción, Procesamiento Estructurado, Procesamiento Vectorial y Carga. Se detallaron las tareas para la etapa de Extracción ("Obtener Lista de Fuentes" y "Descargar/Scrapear Contenido"), se definió la lista de fuentes de datos en una variable, se propusieron y confirmaron herramientas específicas (`requests`, `BeautifulSoup`), se implementó la lógica de descarga/scraping en la clase `ExtractionLogic`, se definió la estructura de la tarea de CrewAI para la extracción, se definió el Agente de CrewAI para la Extracción, y se definió la Herramienta de CrewAI que envuelve la clase `ExtractionLogic`. Se definió con éxito el Agente de Procesamiento Estructurado y el Agente de Procesamiento Vectorial. Se detallaron las tareas para el Agente de Procesamiento Vectorial (Chunking de Texto, Generación de Embeddings, Preparación de Metadatos), se implementó la función para Chunking y Generación de Embeddings, y se asignó la Herramienta de Chunking y Generación de Embeddings al Agente de Procesamiento Vectorial.
    *   Se cargó el contenido del archivo "GEMINI.md" al contexto del notebook.
    *   Se revisó el estado actual del proyecto y se propuso un plan de desarrollo en 8 pasos.
    *   **Paso 1 del Plan (Refinar la Lógica de Extracción) completado:** Se modificó y mejoró la clase `ExtractionLogic` para buscar y descargar archivos enlazados desde páginas HTML.
    *   **Paso 2 del Plan (Diseñar e Implementar el Agente y Tareas de Procesamiento Estructurado) completado:** Se definieron las tareas específicas para el agente de procesamiento estructurado y se implementó la herramienta `extract_structured_data` utilizando el modelo Mistral para extraer datos según los esquemas de Supabase.
    *   **Paso 3 del Plan (Refinar e Implementar las Tareas del Agente de Procesamiento Vectorial) completado:** Se refinó la herramienta `chunk_and_embed_content` para manejar la lectura de contenido desde rutas de archivo y se definió la tarea `prepare_pinecone_metadata_task` para preparar los metadatos de Pinecone.
    *   **Paso 4 del Plan (Diseñar e Implementar el Agente y Tareas de Carga) completado:** Se definió el Agente de Carga, sus tareas (`load_structured_data_to_supabase`, `load_vector_data_to_pinecone`) y sus herramientas (`load_data_to_supabase`, `load_vectors_to_pinecone`).
    *   **Paso 5 del Plan (Integrar Agentes, Tareas y Herramientas en un Crew) completado:** Se definieron las instancias de Agentes y Tareas, se corrigió la asignación de agentes a las tareas y se creó la instancia principal del `Crew` con un proceso secuencial.
    *   **Paso 6 del Plan (Desarrollar Tests) iniciado:** Se definieron los datos de prueba (`data_sources_test`) para facilitar el debugging y testing de las etapas del pipeline. Se modificó la clase `ExtractionTasks` para que la tarea de extracción reciba la lista de fuentes como input. Se corrigió la definición de la herramienta `extraction_tool` para que use el decorador `@tool` de `crewai.tools` y acepte la lista de fuentes como input. Se preparó la ejecución del Crew con los datos de prueba y `verbose=True` para realizar la primera prueba controlada.
    * Todavia no se pudo realizar el primer test, hay un problema: El problema actual es conseguir crear un objeto "LLM" de crewai para usar con agentes del modelo mistralai/Mistral-7B-Instruct-v0.3 desplegado localmente en Google Colab con transformers. CrewAI requiere APIs HTTP (formato OpenAI) pero el modelo está cargado como objeto Python, causando error "model not supported by any provider". Opciones: usar vLLM como servidor local, HuggingFacePipeline de LangChain, o API remota de HF. Necesario: convertir modelo local en formato API compatible para integración con agentes.
*   * *

In [ ]:
%pip install -q -U supabase pinecone sentence-transformers crewai 'crewai[tools]' langchain_community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.2/40.2 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.2/40.2 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 4.4 MB

In [ ]:
from google.colab import userdata
from supabase import create_client, Client

# Asegúrate de haber guardado tus secretos "SUPABASE_URL" y "SUPABASE_API" en Colab Secrets
supabase_url = userdata.get("SUPABASE_URL")
supabase_key = userdata.get("SUPABASE_API")

if supabase_url and supabase_key:
    try:
        supabase: Client = create_client(supabase_url, supabase_key)
        print("Conexión a Supabase establecida con éxito.")

    except Exception as e:
        print(f"Error al conectar a Supabase: {e}")
        print("Verifica que los secretos 'SUPABASE_URL' y 'SUPABASE_API' sean correctos.")

else:
    if not supabase_url:
        print("Error: El secreto 'SUPABASE_URL' no se encontró.")
        print("Asegúrate de haber guardado la URL de tu proyecto Supabase en Colab Secrets con ese nombre.")
    if not supabase_key:
        print("Error: El secreto 'SUPABASE_API' no se encontró.")
        print("Asegúrate de haber guardado tu clave API anon de Supabase en Colab Secrets con ese nombre.")

Conexión a Supabase establecida con éxito.


In [ ]:
from pinecone import Pinecone
from google.colab import userdata

# Asegúrate de haber guardado tu secreto "PINECONE_API" en Colab Secrets
pinecone_api_key = userdata.get("PINECONE_API")

if pinecone_api_key:
    try:
        pc = Pinecone(api_key=pinecone_api_key)
        print("Conexión a Pinecone establecida con éxito.")
        # Opcional: Listar índices para verificar la conexión
        # print("Índices existentes en Pinecone:", pc.list_indexes())

    except Exception as e:
        print(f"Error al conectar a Pinecone: {e}")
        print("Verifica que el secreto 'PINECONE_API' sea correcto.")

else:
    print("Error: El secreto 'PINECONE_API' no se encontró.")
    print("Asegúrate de haber guardado tu clave API de Pinecone en Colab Secrets con ese nombre.")

Conexión a Pinecone establecida con éxito.


In [ ]:
from sentence_transformers import SentenceTransformer

# Cargar el modelo de embeddings
embedding_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
print("Modelo de embeddings 'sentence-transformers/all-MiniLM-L6-v2' cargado con éxito.")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Modelo de embeddings 'sentence-transformers/all-MiniLM-L6-v2' cargado con éxito.


In [ ]:
# Leer el contenido de Fuentes_Inverbot_Pipeline.md
with open("/content/drive/MyDrive/Colab Notebooks/Inverbot_Pipeline_Proyecto/Fuentes_Inverbot_Pipeline.md", "r") as f:
    fuentes_content = f.read()

# Leer el contenido de Estructura_Datos_Inverbot (1).md
with open("/content/drive/MyDrive/Colab Notebooks/Inverbot_Pipeline_Proyecto/Estructura_Datos_Inverbot (1).md", "r") as f:
    estructura_content = f.read()

print("Contenido de Fuentes_Inverbot_Pipeline.md:")
print(fuentes_content)
print("\nContenido de Estructura_Datos_Inverbot (1).md:")
print(estructura_content)

Contenido de Fuentes_Inverbot_Pipeline.md:
## **Fuentes de Datos y Estrategia de Extracción/Carga para CrewAI**

Esta tabla detalla cómo tus agentes CrewAI adquirirán el contenido, lo procesarán y lo cargarán en tu base de datos estructurada (Supabase) y tu base de datos vectorial (Pinecone), incluyendo los enlaces de fuente que has proporcionado.

---

| Categoría de Datos | Fuente Principal (Usuario indica) | Tipo de Contenido Adquirido | Procesamiento por la IA | Tipo de Base de Datos para Carga |
| :---- | :---- | :---- | :---- | :---- |
| **Balances de Empresas** | BVA \- página de emisores https://www.bolsadevalores.com.py/emisores/itti-s-a-e-c-a/ | EXCELS de balances, PDFs de prospectos, análisis de riesgo y hechos relevantes, texto de metadatos. | **Estructurar:** Métricas financieras, fechas de corte, moneda, calificaciones, otras\_metricas\_jsonb. \<br\> **Vectorizar:** Texto completo de PDFs (prospectos, análisis de riesgo, hechos relevantes), resúmenes, metadatos identifica

In [ ]:
%pip install -q transformers torch

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_name = "mistralai/Mistral-7B-Instruct-v0.3"

try:
    # Cargar el tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    # Cargar el modelo
    # Puedes especificar device_map="auto" para usar la GPU si está disponible
    model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.bfloat16, device_map="auto")

    print(f"Modelo '{model_name}' desplegado con éxito desde Hugging Face.")

except Exception as e:
    print(f"Error al desplegar el modelo '{model_name}' desde Hugging Face: {e}")
    print("Verifica el nombre del modelo y tu entorno.")

tokenizer_config.json:   0%|          | 0.00/141k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/601 [00:00<?, ?B/s]

...

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.55G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

Modelo 'mistralai/Mistral-7B-Instruct-v0.3' desplegado con éxito desde Hugging Face.


In [ ]:
# Definición de las fuentes de datos para la etapa de Extracción
# Se utiliza una lista de diccionarios para mantener la modularidad y detallar el tipo de contenido esperado.
data_sources = [
    {
        "category": "Balances de Empresas",
        "url": "https://www.bolsadevalores.com.py/listado-de-emisores/",
        "content_type": ["EXCEL", "PDF", "TEXT"], # Incluye PDFs, EXCELS y texto de metadatos
        "description": "Página de listado de emisores de la BVA, contiene un listado de emisores, cuando se selecciona un emisor en específico, se pueden encontrar balances, prospectos, análisis de riesgo y hechos relevantes del emisor en cuestión."
    },
    {
        "category": "Movimientos Diarios",
        "url": "https://www.bolsadevalores.com.py/informes-diarios/",
        "content_type": ["JSON"], # Se espera un feed JSON
        "description": "Informes diarios de la BVA con movimientos del mercado."
    },
    {
        "category": "Volumen Mensual",
        "url": "https://www.bolsadevalores.com.py/informes-mensuales/",
        "content_type": ["TEXT", "PDF", "JSON"], # Metadatos, PDFs e info de JSONs
        "description": "Informes mensuales de la BVA, incluyendo PDFs y datos estructurados."
    },
    {
        "category": "Resumen Anual",
        "url": "https://www.bolsadevalores.com.py/informes-anuales/",
        "content_type": ["TEXT", "PDF"], # Metadatos y PDFs
        "description": "Informes anuales de la BVA en formato PDF."
    },
     {
        "category": "Contexto Macroeconómico",
        "url": "https://www.bcp.gov.py/", # URL general, se necesitará scrapear o buscar enlaces específicos
        "content_type": ["TEXT", "PDF", "EXCEL", "PPT"], # Noticias, datos SITUFIN, informes/estudios
        "description": "Sitio web del Banco Central del Paraguay (BCP) para datos macroeconómicos."
    },
    {
        "category": "Estadísticas Sociales",
        "url": "https://www.ine.gov.py/vt/publicacion.php/", # URL general, se necesitará scrapear o buscar enlaces específicos
        "content_type": ["PDF", "EXCEL", "PPT", "TEXT"], # Informes, estudios y metadatos
        "description": "Portal del Instituto Nacional de Estadística (INE) para publicaciones y datos sociales."
    },
    {
        "category": "Contratos Públicos",
        "url": "https://www.contrataciones.gov.py/", # URL general, se necesitará scrapear feed/página
        "content_type": ["TEXT"], # Datos de licitaciones/contratos directamente de la web
        "description": "Portal de la Dirección Nacional de Contrataciones Públicas (DNCP) para datos de licitaciones y contratos."
    },
     {
        "category": "Datos de Inversión",
        "url": "https://www.dnit.gov.py/web/portal-institucional/invertir-en-py",
        "content_type": ["TEXT", "PNG", "PDF"], # Metadatos, PNGs y PDFs
        "description": "Sección del portal del DNIT con información para invertir en Paraguay."
    },
    {
        "category": "Informes Financieros (DNIT)",
        "url": "https://www.dnit.gov.py/web/portal-institucional/informes-financieros",
        "content_type": ["TEXT", "PDF"], # Metadatos y PDFs
        "description": "Sección del portal del DNIT con informes financieros."
    }
]

# Puedes imprimir la lista para verificar su contenido
# display(data_sources)

In [ ]:
import requests
from bs4 import BeautifulSoup
from crewai import Task
from typing import List, Dict, Any

# Aunque CrewAI usa un Agent para ejecutar Tasks, definiremos la Task primero.
# El Agent de Extracción usará esta tarea.

class ExtractionTasks:
    def __init__(self, agent):
        self.agent = agent
        # Eliminamos data_sources del constructor ya que la tarea lo recibirá como input

    def download_and_scrape_content(self):
        return Task(
            description=(
                "Iterate through the provided data sources, download or scrape content based on type.\n"
                "Handle different content types: JSON (fetch data), TEXT (scrape HTML), PDF/EXCEL/PNG (download file).\n"
                "Return a list of dictionaries, each containing source info and raw content/file path.\n"
                "Input should be a list of dictionaries, each with 'category', 'url', and 'content_type'."
            ),
            expected_output=(
                "A list of dictionaries, where each dictionary includes:\n"
                "- 'source_category': The category of the data source.\n"
                "- 'source_url': The URL from which the content was obtained.\n"
                "- 'content_type': The type of content ('JSON', 'TEXT', 'PDF', 'EXCEL', 'PNG').\n"
                "- 'raw_content': The raw content (for JSON/TEXT) or local file path (for PDF/EXCEL/PNG)."
            ),
            agent=self.agent,
            # Ahora la tarea espera un input, que se pasará al ejecutar el crew
            # input_schema=..., # Podríamos definir un schema para el input si fuera necesario
            callback=lambda output: print(f"Extraction Task Output: {output}") # Opcional: ver el output intermedio
        )

# Nota: La implementación real de la lógica de descarga y scraping
# se realizará dentro del método `_execute` del agente o mediante herramientas asociadas a la tarea.
# La herramienta de extracción (ExtractionTool) necesitará ser ajustada para recibir
# la lista de fuentes como input del Agente/Tarea, en lugar de depender de una instancia
# pre-creada con data_sources fijas.

In [ ]:
import os
import requests
from bs4 import BeautifulSoup
import json # Importar json para manejar datos JSON
from urllib.parse import urljoin # Importar urljoin para manejar URLs relativas

# Directorio temporal para guardar archivos descargados
DOWNLOAD_DIR = "/tmp/inverbot_downloads"
os.makedirs(DOWNLOAD_DIR, exist_ok=True)

class ExtractionLogic:
    def __init__(self, data_sources):
        self.data_sources = data_sources

    def _download_file(self, url, category):
        """Downloads a file from a URL and saves it to a temporary directory."""
        try:
            response = requests.get(url, stream=True)
            response.raise_for_status() # Raise an HTTPError for bad responses (4xx or 5xx)

            # Determine file extension from URL or headers
            filename = url.split('/')[-1] or f"downloaded_file_{len(os.listdir(DOWNLOAD_DIR))}"
            if '.' not in filename:
                 # Try to get content type from headers to guess extension
                 content_type = response.headers.get('Content-Type')
                 if content_type:
                     if 'pdf' in content_type:
                         filename += '.pdf'
                     elif 'excel' in content_type or 'spreadsheetml' in content_type:
                         filename += '.xlsx' # Assuming xlsx for modern excel
                     elif 'png' in content_type:
                         filename += '.png'
                     elif 'presentation' in content_type:
                         filename += '.pptx' # Assuming pptx for modern powerpoint
                     # Add more types as needed

            file_path = os.path.join(DOWNLOAD_DIR, filename)

            with open(file_path, 'wb') as f:
                for chunk in response.iter_content(chunk_size=8192):
                    f.write(chunk)

            print(f"Downloaded file from {url} to {file_path}")
            return file_path

        except requests.exceptions.RequestException as e:
            print(f"Error downloading file from {url}: {e}")
            return None
        except Exception as e:
            print(f"An unexpected error occurred while downloading {url}: {e}")
            return None

    def _scrape_html(self, url, category):
        """Scrapes text content and finds file links from an HTML page."""
        try:
            response = requests.get(url)
            response.raise_for_status()

            soup = BeautifulSoup(response.content, 'html.parser')

            # Find potential file links (PDF, EXCEL, PNG, PPT)
            file_links = []
            for link in soup.find_all('a', href=True):
                href = link['href']
                full_url = urljoin(url, href) # Handle relative URLs
                # Simple check for file extensions - could be improved
                if any(full_url.lower().endswith(ext) for ext in ['.pdf', '.xls', '.xlsx', '.png', '.ppt', '.pptx']):
                    file_links.append(full_url)

            # Extract text, removing script and style elements
            for script_or_style in soup(["script", "style"]):
                script_or_style.extract()
            text = soup.get_text()

            # Clean up whitespace
            text = ' '.join(text.split())

            print(f"Scraped HTML from {url}")
            return {"text_content": text, "file_links": file_links}

        except requests.exceptions.RequestException as e:
            print(f"Error scraping HTML from {url}: {e}")
            return {"text_content": None, "file_links": []}
        except Exception as e:
            print(f"An unexpected error occurred while scraping {url}: {e}")
            return {"text_content": None, "file_links": []}


    def _fetch_json(self, url, category):
        """Fetches and returns JSON data from a URL."""
        try:
            response = requests.get(url)
            response.raise_for_status()
            json_data = response.json()
            print(f"Fetched JSON from {url}")
            return json_data

        except requests.exceptions.RequestException as e:
            print(f"Error fetching JSON from {url}: {e}")
            return None
        except json.JSONDecodeError:
            print(f"Error decoding JSON from {url}: Invalid JSON response.")
            return None
        except Exception as e:
            print(f"An unexpected error occurred while fetching JSON from {url}: {e}")
            return None


    def run_extraction(self):
        """Iterates through data sources and extracts content."""
        extracted_data = []
        for source in self.data_sources:
            category = source["category"]
            url = source["url"]
            content_types = source["content_type"]

            print(f"Processing source: {category} from {url}")

            # Prioritize JSON if expected
            if "JSON" in content_types:
                 raw_content = self._fetch_json(url, category)
                 if raw_content is not None:
                      extracted_data.append({
                          "source_category": category,
                          "source_url": url,
                          "content_type": "JSON",
                          "raw_content": raw_content # JSON data
                      })
            # Otherwise, attempt to scrape HTML and look for files if TEXT/file types are expected
            elif any(ct in content_types for ct in ["TEXT", "PDF", "EXCEL", "PPT", "PNG"]):
                 scraped_result = self._scrape_html(url, category)
                 if scraped_result.get("text_content") is not None:
                      extracted_data.append({
                          "source_category": category,
                          "source_url": url,
                          "content_type": "TEXT", # Store the scraped page text
                          "raw_content": scraped_result["text_content"]
                      })

                 # Download found files
                 for file_url in scraped_result.get("file_links", []):
                     file_path = self._download_file(file_url, category)
                     if file_path:
                         # Determine file type from extension
                         file_ext = os.path.splitext(file_path)[1].lstrip('.').upper()
                         extracted_data.append({
                              "source_category": category,
                              "source_url": file_url, # The URL of the actual file
                              "content_type": file_ext, # e.g., 'PDF', 'XLSX', 'PNG'
                              "raw_content": file_path # The local path to the downloaded file
                         })

            else:
                print(f"Warning: No extraction logic defined for content types {content_types} for source {category} from {url}. Skipping.")


        return extracted_data

# Example of how you might use this logic (without CrewAI Agent yet):
# extraction_handler = ExtractionLogic(data_sources)
# extracted_raw_data = extraction_handler.run_extraction()
# print("\n--- Extracted Data Summary ---")
# for item in extracted_raw_data:
#     print(f"Source: {item['source_category']}, Type: {item['content_type']}, URL: {item['source_url']}")
#     if item['content_type'] in ['TEXT', 'JSON']:
#         print(f"  Content snippet: {str(item['raw_content'])[:200]}...")
#     else: # File path
#          print(f"  Local Path: {item['raw_content']}")

In [ ]:
from crewai import Agent
# Asumimos que el modelo Mistral ya fue cargado y está disponible como 'model' y 'tokenizer'
# desde las celdas anteriores de Hugging Face.
# crewAI necesita un 'Language Model' compatible con su framework.
# Podemos adaptar el modelo cargado o usar una envoltura si es necesario.

# Si estás usando la integración con langchain, podrías necesitar un wrapper.
# Por ejemplo:
# from langchain_community.llms import HuggingFacePipeline
# from transformers import pipeline
#
# pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=512)
# mistral_llm = HuggingFacePipeline(pipeline=pipe)

# Sin un wrapper específico de langchain para un modelo custom cargado directamente,
# crewAI podría requerir un formato particular o una integración custom.
# Para simplificar y seguir avanzando con el diseño, definiremos el agente
# y abordaremos la integración específica del modelo cargado en CrewAI
# al construir el Crew completo, o usaremos un modelo accesible vía API si la integración
# directa es compleja.

#**Vamos a definir el Agente de Extracción:**
extraction_agent = Agent(
  role='Data Extractor',
  goal='Obtain raw content from various web sources based on provided URLs and content types.',
  backstory="""You are an expert in web scraping, API calls, and file downloading.
  You can efficiently retrieve content from diverse online sources, including
  HTML pages, JSON feeds, PDFs, and Excel files.""",
  # tools=[tool_for_downloading, tool_for_scraping, tool_for_json], # Herramientas se añadirán aquí
  verbose=True,
  allow_delegation=False, # Este agente se enfoca solo en la extracción
  # crewai espera un objeto LLM compatible, como los de Langchain o LlamaIndex.
  # La integración directa con el modelo 'model' cargado con transformers
  # puede requerir una envoltura custom o una herramienta que use el modelo.
  # Por ahora, definimos el agente. La asignación del LLM se hará al construir el Crew.
  # Si usamos herramientas que internamente usan el modelo Mistral, no necesitamos asignarlo directamente al agente.
  # Para esta etapa de diseño, nos enfocamos en el rol y tareas.
)

print("Agente de Extracción definido.")

Agente de Extracción definido.


In [ ]:
from crewai import Agent
# Importar el decorador @tool de crewai.tools
from crewai.tools import tool
from typing import List, Dict, Any # Importar tipos para anotaciones

# Asegúrate de que el modelo Mistral ya fue cargado y está disponible como 'model' y 'tokenizer'
# desde las celdas anteriores de Hugging Face.
# crewAI necesita un 'Language Model' compatible con su framework.
# Podemos adaptar el modelo cargado o usar una envoltura si es necesario.

# Si estás usando la integración con langchain, podrías necesitar un wrapper.
# Por ejemplo:
# from langchain_community.llms import HuggingFacePipeline
# from transformers import pipeline
#
# pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=512)
# mistral_llm = HuggingFacePipeline(pipeline=pipe)

# Sin un wrapper específico de langchain para un modelo custom cargado directamente,
# crewAI podría requerir un formato particular o una integración custom.
# Para simplificar y seguir avanzando con el diseño, definiremos el agente
# y abordaremos la integración específica del modelo cargado en CrewAI
# al construir el Crew completo, o usaremos un modelo accesible vía API si la integración
# directa es compleja.

# Asegúrate de que la clase ExtractionLogic esté definida en una celda anterior
# from .extraction_logic import ExtractionLogic # Si estuviera en otro archivo

# Definimos la herramienta de extracción usando el decorador @tool de crewai.tools
@tool
def extraction_tool(sources_list: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
    """
    A tool to extract raw content from a provided list of data sources.
    Input is a list of dictionaries, each with 'category', 'url', and 'content_type'.
    This version is adapted to accept input directly from the task for testing purposes.
    """
    print(f"Extraction Tool received sources_list with {len(sources_list)} items.")
    # Creamos una nueva instancia de ExtractionLogic con los datos de entrada
    temp_extraction_handler = ExtractionLogic(data_sources=sources_list)
    return temp_extraction_handler.run_extraction()


#**Vamos a definir el Agente de Extracción:**
extraction_agent = Agent(
  role='Data Extractor',
  goal='Obtain raw content from various web sources based on provided URLs and content types.',
  backstory="""You are an expert in web scraping, API calls, and file downloading.
  You can efficiently retrieve content from diverse online sources, including
  HTML pages, JSON feeds, PDFs, and Excel files.""",
  tools=[extraction_tool], # Asignamos la herramienta definida con @tool
  verbose=True,
  allow_delegation=False, # Este agente se enfoca solo en la extracción
  # crewai espera un objeto LLM compatible, como los de Langchain o LlamaIndex.
  # La integración directa con el modelo 'model' cargado con transformers
  # puede requerir una envoltura custom o una herramienta que use el modelo.
  # Por ahora, definimos el agente. La asignación del LLM se hará al construir el Crew.
  # Si usamos herramientas que internamente usan el modelo Mistral, no necesitamos asignarlo directamente al agente.
  # Para esta etapa de diseño, nos enfocamos en el rol y tareas.
  llm=mistral_llm_for_crewai # Asegúrate de que mistral_llm_for_crewai esté definido
)

print("Agente de Extracción definido con herramienta modificada (usando @tool).")

Agente de Extracción definido con herramienta modificada (usando @tool).


In [ ]:
from crewai import Agent
# Asegúrate de que el modelo Mistral 'model' cargado desde Hugging Face esté disponible.
# Si no, necesitarás un wrapper compatible con CrewAI, como HuggingFacePipeline de langchain.

from langchain_community.llms import HuggingFacePipeline
from transformers import pipeline

# Creamos un pipeline de transformers con el modelo y tokenizer cargados
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=512, trust_remote_code=True) # Añadido trust_remote_code=True por si acaso

# Envolvemos el pipeline en un objeto compatible con Langchain/CrewAI
mistral_llm_for_crewai = HuggingFacePipeline(pipeline=pipe)

# Define el Agente de Procesamiento Estructurado
structured_processing_agent = Agent(
    role='Structured Data Processor',
    goal='Extract and structure key information from raw content according to the Supabase schema.',
    backstory="""You are an expert in parsing diverse data formats (text, JSON, documents)
    and transforming them into structured records that match predefined database schemas.
    You use advanced language models to intelligently identify and extract relevant data points.""",
    # tools=[parsing_tool_for_pdfs, json_parser_tool, text_structuring_tool], # Herramientas se añadirán aquí
    verbose=True,
    allow_delegation=False, # Este agente se enfoca solo en el procesamiento estructurado
    # Asigna el modelo Mistral cargado o su wrapper compatible aquí
    llm=mistral_llm_for_crewai # Usamos el wrapper compatible con CrewAI
)

print("Agente de Procesamiento Estructurado definido.")

Agente de Procesamiento Estructurado definido.


/tmp/ipython-input-13-1763727686.py:12: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  mistral_llm_for_crewai = HuggingFacePipeline(pipeline=pipe)


In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
# Asegúrate de que el modelo de embeddings `embedding_model` ya esté cargado
# from sentence_transformers import SentenceTransformer
# embedding_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')


def chunk_text_and_generate_embeddings(text: str, chunk_size: int = 500, chunk_overlap: int = 50):
    """
    Chunks the input text and generates embeddings for each chunk.

    Args:
        text: The input text string.
        chunk_size: The maximum size of each text chunk.
        chunk_overlap: The number of characters to overlap between chunks.

    Returns:
        A list of dictionaries, where each dictionary contains:
        - 'chunk_text': The text of the chunk.
        - 'embedding': The embedding vector for the chunk.
    """
    if not text:
        return []

    # Inicializar el text splitter
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
        is_separator_regex=False, # Using standard separators
    )

    # Dividir el texto en chunks
    chunks = text_splitter.split_text(text)

    # Generar embeddings para cada chunk
    chunk_embeddings = []
    for i, chunk in enumerate(chunks):
        try:
            # Generar el embedding usando el modelo cargado globalmente
            # Asegúrate de que `embedding_model` esté disponible en este scope
            embedding = embedding_model.encode(chunk).tolist() # Convert to list for easier handling/storage

            chunk_embeddings.append({
                'chunk_text': chunk,
                'embedding': embedding,
                'chunk_id': i # Simple index as chunk ID, can be improved later
            })
        except Exception as e:
            print(f"Error generating embedding for chunk {i}: {e}")
            # Optionally, decide how to handle errors: skip chunk, log, etc.
            continue # Skip this chunk if embedding generation fails

    return chunk_embeddings

# Example Usage (optional, for testing):
# sample_text = "This is a long document that needs to be chunked for vectorization. Embeddings will be generated for each piece."
# embeddings_with_chunks = chunk_text_and_generate_embeddings(sample_text)
# print(f"Generated {len(embeddings_with_chunks)} chunks with embeddings.")
# # display(embeddings_with_chunks) # Use display if the output is large

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from crewai import Agent
# No necesitamos importar Tool o BaseTool directamente si usamos el decorador @tool
# from langchain_core.tools import BaseTool
# from langchain.tools import tool
# from crewai_tools import tool
from crewai.tools import tool # Importar el decorador @tool desde crewai.tools
from typing import Any, Text, Dict, List, Union
import os # Importar os para verificar si una ruta existe

# Asegúrate de que el modelo de embeddings `embedding_model` ya esté cargado
# from sentence_transformers import SentenceTransformer
# embedding_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

# Asegúrate de que el modelo Mistral LLM (para razonamiento del agente)
# y su wrapper compatible con CrewAI estén disponibles.
# from langchain_community.llms import HuggingFacePipeline
# from transformers import pipeline
# pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=512, trust_remote_code=True)
# mistral_llm_for_crewai = HuggingFacePipeline(pipeline=pipe)

# Reutilizamos la función auxiliar para leer archivos definida en la celda anterior (cb692459)
# def read_file_content(file_path: str) -> Union[str, None]:
#     """Reads the content of a file given its path."""
#     try:
#         with open(file_path, 'r', encoding='utf-8') as f:
#             return f.read()
#     except Exception as e:
#         print(f"Error reading file {file_path}: {e}")
#         return None


# Definimos la función para chunking y embeddings (asegúrate de que esta función esté definida en una celda anterior y ejecutada)
# def chunk_text_and_generate_embeddings(text: str, chunk_size: int = 500, chunk_overlap: int = 50):
#     """
#     Chunks the input text and generates embeddings for each chunk.
#     ... (código de la función) ...

@tool
def chunk_and_embed_content(content: Union[str, Dict[str, Any]]) -> List[Dict[str, Any]]:
    """
    Takes content (text or file path), chunks it, and generates embeddings for each chunk
    using a pre-loaded model. If the content is a file path, it reads the file first.
    Input should be the text content or a valid local file path string to process.
    Handles JSON content by converting it to a string.
    """
    print(f"Using chunk_and_embed_content tool on content (first 100 chars or type): {str(content)[:100]}...")

    processed_text = None
    if isinstance(content, str):
        # Check if it's likely a file path
        if os.path.exists(content):
            print(f"Content is a file path, reading file: {content}")
            # Assuming read_file_content is defined elsewhere and handles different file types or we need a more specific tool here
            # For now, assuming text files or we need a dedicated tool to convert docs to text first.
            # Let's assume for now that if it's a file path, it's a text file or a format we can directly read as text.
            # A more robust solution would involve a separate tool for PDF/EXCEL to text conversion.
            try:
                with open(content, 'r', encoding='utf-8') as f:
                    processed_text = f.read()
            except Exception as e:
                print(f"Error reading file {content} for chunking: {e}")
                return [] # Return empty list if file reading fails
        else:
            processed_text = content # Assume it's raw text
    elif isinstance(content, dict):
         # Convert dictionary (e.g., JSON content) to string
         processed_text = json.dumps(content, indent=2)
    else:
        print(f"Unsupported content type for chunking: {type(content)}")
        return []


    if not processed_text:
        print("No text content to chunk and embed.")
        return []

    try:
        # Llamar a la función existente para chunking y embeddings
        # Asegúrate de que chunk_text_and_generate_embeddings esté definida y embedding_model cargado
        # Note: chunk_text_and_generate_embeddings currently expects a string.
        # We are ensuring processed_text is a string before calling it.
        chunk_embeddings = chunk_text_and_generate_embeddings(processed_text)
        print(f"Generated {len(chunk_embeddings)} chunks with embeddings.")
        # Devolvemos la lista de diccionarios con chunks y embeddings
        return chunk_embeddings
    except Exception as e:
        print(f"An error occurred during chunking and embedding: {e}")
        return [] # Devolver lista vacía en caso de error

# Define el Agente de Procesamiento Vectorial
vectorial_processing_agent = Agent(
    role='Vectorial Data Processor',
    goal='Generate text embeddings and prepare data with relevant metadata for Pinecone.',
    backstory="""You are an expert in natural language processing and vector databases.
    You can effectively chunk text, generate high-quality embeddings, and structure
    data with necessary metadata for efficient storage and retrieval in Pinecone.""",
    tools=[chunk_and_embed_content], # Asignamos la herramienta actualizada
    verbose=True,
    allow_delegation=False, # Este agente se enfoca solo en el procesamiento vectorial
    # Asigna el modelo Mistral cargado o su wrapper compatible aquí para el razonamiento del agente
    llm=mistral_llm_for_crewai # Usamos el wrapper compatible con CrewAI para el LLM
)

print("Agente de Procesamiento Vectorial definido con herramientas.")

Agente de Procesamiento Vectorial definido con herramientas.


In [ ]:
from crewai import Task

# Asegúrate de que el Agente de Procesamiento Estructurado (structured_processing_agent)
# ya esté definido en una celda anterior.

class StructuredProcessingTasks:
    def __init__(self, agent):
        self.agent = agent

    def process_financial_reports(self):
        return Task(
            description=(
                "Analyze raw content from financial reports (PDF text or scraped text related to balances/reports)."
                "Extract key financial metrics, dates, currency, ratings, and other relevant structured data points "
                "based on the 'Resumen_Informe_Financiero' and 'Informe_General' Supabase schemas."
                "Handle potential variations in report formats and missing data gracefully."
                "The output should be a structured dictionary or list of dictionaries conforming to the schema."
            ),
            expected_output=(
                "A dictionary or list of dictionaries, each representing a record for the Supabase tables "
                "'Resumen_Informe_Financiero' and/or 'Informe_General', containing extracted structured data "
                "(e.g., {'fecha_corte_informe': 'YYYY-MM-DD', 'activos_totales': 12345.67, 'moneda_informe': 1, ...}). "
                "Include necessary foreign keys like 'id_emisor' if identifiable from the source or context."
            ),
            agent=self.agent,
            # Tools will be assigned to the agent or task later.
        )

    def process_daily_movements_json(self):
        return Task(
            description=(
                "Parse the raw JSON content from daily market movement reports."
                "Extract all relevant fields such as date, quantity, price, instrument ID, issuer ID, currency, etc., "
                "according to the 'Movimiento_Diario_Bolsa' Supabase schema."
                "Ensure data types are correct for database insertion."
            ),
            expected_output=(
                "A list of dictionaries, where each dictionary represents a record for the 'Movimiento_Diario_Bolsa' "
                "Supabase table, containing parsed JSON data (e.g., [{'fecha_operacion': 'YYYY-MM-DD', 'cantidad_operacion': 1000.00, ...}])."
            ),
            agent=self.agent,
            # Tools will be assigned to the agent or task later.
        )

    def process_macroeconomic_data(self):
        return Task(
            description=(
                "Analyze raw content (text from pages, content from reports/spreadsheets) related to macroeconomic data."
                "Identify and extract specific macroeconomic indicators, their values, dates, units of measure, frequency, "
                "and source links, mapping them to the 'Dato_Macroeconomico' Supabase schema."
                "Handle different formats and potential ambiguity in text."
            ),
            expected_output=(
                "A dictionary or list of dictionaries, each representing a record for the 'Dato_Macroeconomico' Supabase table, "
                "containing extracted macroeconomic data (e.g., {'indicador_nombre': 'Inflacion', 'fecha_dato': 'YYYY-MM-DD', 'valor_numerico': 5.2, ...})."
                "Include relevant foreign keys ('unidad_medida', 'id_frecuencia', 'id_moneda', 'id_emisor' if applicable) by looking up or inferring IDs."
            ),
            agent=self.agent,
            # Tools will be assigned to the agent or task later.
        )

    def process_public_contracts(self):
        return Task(
            description=(
                "Extract structured information about public tenders and contracts from raw text content."
                "Identify key fields such as title, contracting entity, awarded amount, currency, "
                "and award date, mapping them to the 'Licitacion_Contrato' Supabase schema."
                "Handle variations in text structure and extract accurate numerical and date values."
            ),
            expected_output=(
                "A dictionary or list of dictionaries, each representing a record for the 'Licitacion_Contrato' Supabase table, "
                "containing extracted contract data (e.g., {'titulo': 'Adquisicion de Servicios', 'monto_adjudicado': 150000.00, 'fecha_adjudicacion': 'YYYY-MM-DD', ...})."
                "Include 'id_emisor_adjudicado' if the awarded entity can be identified and mapped to an existing emitter."
            ),
            agent=self.agent,
            # Tools will be assigned to the agent or task later.
        )

    def process_general_reports_metadata(self):
         return Task(
            description=(
                "Extract general metadata from various reports and documents (annual reports, investment data, etc.) "
                "that don't fit specifically into financial, macroeconomic, or contract schemas, "
                "but provide context about the document itself."
                "Map this information to the 'Informe_General' Supabase schema fields like title, publication date, summary (if available)."
            ),
            expected_output=(
                "A dictionary or list of dictionaries, each representing a record for the 'Informe_General' Supabase table, "
                "containing general report metadata (e.g., {'titulo_informe': 'Informe Anual 2023', 'fecha_publicacion': 'YYYY-MM-DD', 'resumen_informe': '...'})."
                "Include relevant foreign keys ('id_emisor', 'id_tipo_informe', 'id_frecuencia', 'id_periodo') by looking up or inferring IDs."
            ),
            agent=self.agent,
            # Tools will be assigned to the agent or task later.
         )


# Example of how to instantiate these tasks (requires structured_processing_agent to be defined):
# structured_tasks = StructuredProcessingTasks(structured_processing_agent)
# financial_task = structured_tasks.process_financial_reports()
# daily_movements_task = structured_tasks.process_daily_movements_json()
# macroeconomic_task = structured_tasks.process_macroeconomic_data()
# public_contracts_task = structured_tasks.process_public_contracts()
# general_reports_task = structured_tasks.process_general_reports_metadata()

print("Tareas para el Agente de Procesamiento Estructurado definidas.")

Tareas para el Agente de Procesamiento Estructurado definidas.


In [ ]:
from crewai.tools import tool
from typing import Any, Dict, List, Union

# Asegúrate de que el wrapper del modelo Mistral para CrewAI (mistral_llm_for_crewai) esté disponible
# from langchain_community.llms import HuggingFacePipeline
# from transformers import pipeline
# pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=512, trust_remote_code=True)
# mistral_llm_for_crewai = HuggingFacePipeline(pipeline=pipe)

# Puedes necesitar una función auxiliar para leer el contenido de archivos si la entrada es una ruta de archivo
def read_file_content(file_path: str) -> Union[str, None]:
    """Reads the content of a file given its path."""
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            return f.read()
    except Exception as e:
        print(f"Error reading file {file_path}: {e}")
        return None

# Definimos la herramienta usando el decorador @tool
@tool
def extract_structured_data(
    content: Union[str, Dict[str, Any]],
    schema_name: str, # e.g., "Resumen_Informe_Financiero", "Movimiento_Diario_Bolsa"
    schema_description: str # Description or structure of the target schema
) -> Union[Dict[str, Any], List[Dict[str, Any]], None]:
    """
    Analyzes raw content (text, JSON, or file path) and extracts structured data
    based on a provided Supabase schema description.

    Args:
        content: The raw content (string text, dictionary JSON, or file path string).
                 If it's a file path, the tool will attempt to read the file.
        schema_name: The name of the target Supabase table/schema (e.g., "Resumen_Informe_Financiero").
        schema_description: A string describing the target schema structure (column names, types, purpose).

    Returns:
        A dictionary or list of dictionaries containing the extracted structured data,
        formatted according to the specified schema, or None if extraction fails.
    """
    print(f"Using extract_structured_data tool for schema: {schema_name}")

    processed_content = None
    if isinstance(content, str):
        # Check if it's likely a file path (simple check, could be improved)
        if os.path.exists(content):
            print(f"Content is a file path, reading file: {content}")
            processed_content = read_file_content(content)
            if processed_content is None:
                return None # Failed to read file
        else:
            processed_content = content # Assume it's raw text
    elif isinstance(content, dict):
        processed_content = json.dumps(content, indent=2) # Convert JSON dict to string
    else:
        print(f"Unsupported content type: {type(content)}")
        return None

    if not processed_content:
        print("No content to process.")
        return None

    # Use the Mistral LLM (via its wrapper) to perform the extraction
    # Construct a prompt for the LLM
    prompt = f"""
    You are an expert data extraction AI. Your task is to extract structured data
    from the provided content based on the given schema description.

    Target Schema Name: {schema_name}
    Target Schema Description:
    {schema_description}

    Content to Extract From:
    ---
    {processed_content}
    ---

    Extract the data strictly following the schema. Pay attention to data types.
    If a field is not found in the content, set it to None or a suitable default
    (e.g., 0 for numerical fields if appropriate, but prefer None if ambiguous).
    For JSONB fields, structure the extracted data as a JSON object.
    Return the extracted data as a JSON formatted string. If multiple records
    can be extracted (e.g., from a list in JSON or multiple sections in text),
    return a JSON array of objects. If only one record is expected, return a
    single JSON object.

    Ensure the output is valid JSON that can be parsed directly into a Python dictionary or list of dictionaries.
    Only return the JSON output, nothing else.
    """

    try:
        # Use the LLM to generate the structured data
        # The exact method depends on the LLM wrapper's interface
        # Assuming mistral_llm_for_crewai has a method like .invoke() or .generate()
        # You might need to adjust this based on the actual wrapper implementation
        print("Sending prompt to LLM for structured extraction...")
        response = mistral_llm_for_crewai.invoke(prompt) # Or similar method

        # The LLM should return a JSON string. Parse it.
        # Need to handle potential non-JSON output from the LLM
        try:
            extracted_data = json.loads(response)
            print(f"Successfully extracted and parsed data for schema {schema_name}.")
            # print(extracted_data) # Optional: print extracted data for debugging
            return extracted_data
        except json.JSONDecodeError as e:
            print(f"LLM response was not valid JSON: {response[:500]}...")
            print(f"JSON decoding error: {e}")
            return None
        except Exception as e:
            print(f"An unexpected error occurred while parsing LLM response: {e}")
            return None

    except Exception as e:
        print(f"An error occurred while calling the LLM for structured extraction: {e}")
        return None


# This tool can now be assigned to the StructuredProcessingAgent
# structured_processing_agent.tools = [extract_structured_data]

print("Herramienta 'extract_structured_data' definida.")

Herramienta 'extract_structured_data' definida.


In [ ]:
from crewai import Task
from typing import List, Dict, Any

# Asegúrate de que el Agente de Procesamiento Vectorial (vectorial_processing_agent)
# ya esté definido en una celda anterior.

class VectorialProcessingTasks:
    def __init__(self, agent):
        self.agent = agent

    def chunk_and_embed_content_task(self):
         return Task(
             description=(
                 "Receive raw or processed content (text or file path) and use the 'chunk_and_embed_content' tool "
                 "to split it into chunks and generate embeddings for each chunk."
                 "The output should be a list of dictionaries, each containing 'chunk_text', 'embedding', and 'chunk_id'."
             ),
             expected_output=(
                 "A list of dictionaries, where each dictionary has the keys 'chunk_text' (string), "
                 "'embedding' (list of floats), and 'chunk_id' (int)."
             ),
             agent=self.agent,
             # La herramienta 'chunk_and_embed_content' debe estar asignada al agente
             # o a la tarea si usas crewai >= 0.28
             # tools=[chunk_and_embed_content] # Descomentar si asignas herramientas por tarea
         )

    def prepare_pinecone_metadata_task(self):
        return Task(
            description=(
                "Receive a list of text chunks with their embeddings and the original source/structured data information."
                "Prepare the metadata dictionary for each chunk according to the relevant Pinecone index schema "
                "(documentos-informes-vector, noticia-relevante-vector, dato-macroeconomico-vector, licitacion-contrato-vector)."
                "Include all required metadata fields like IDs, dates, titles, etc., looking up or inferring values as needed."
                "Ensure metadata values match the expected types for Pinecone."
            ),
            expected_output=(
                "A list of dictionaries, where each dictionary represents a vector to be inserted into Pinecone, "
                "containing the 'id' (unique string, e.g., source_id-chunk_id), 'values' (the embedding list), "
                "and 'metadata' (a dictionary with keys matching the Pinecone index schema, e.g., "
                "{'id_informe': 123, 'fecha_publicacion': 'YYYY-MM-DD', 'chunk_text': '...'})."
            ),
            agent=self.agent,
            # Herramientas para buscar IDs o inferir metadatos podrían ser necesarias aquí,
            # aunque gran parte de la lógica puede estar en la descripción de la tarea para el LLM.
        )

# Ejemplo de cómo instanciar estas tareas (requiere vectorial_processing_agent definido):
# vectorial_tasks = VectorialProcessingTasks(vectorial_processing_agent)
# chunking_task = vectorial_tasks.chunk_and_embed_content_task()
# metadata_task = vectorial_tasks.prepare_pinecone_metadata_task()

print("Tareas para el Agente de Procesamiento Vectorial definidas.")

Tareas para el Agente de Procesamiento Vectorial definidas.


In [ ]:
from crewai import Agent
# Asegúrate de que el wrapper del modelo Mistral para CrewAI (mistral_llm_for_crewai) esté disponible
# from langchain_community.llms import HuggingFacePipeline
# from transformers import pipeline
# pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=512, trust_remote_code=True)
# mistral_llm_for_crewai = HuggingFacePipeline(pipeline=pipe)

# Define el Agente de Carga
loading_agent = Agent(
    role='Data Loader',
    goal='Load structured data into Supabase and vector data into Pinecone.',
    backstory="""You are an expert in database operations, specifically with Supabase
    and Pinecone. You can efficiently insert and update records, ensuring data
    integrity and handling potential conflicts or existing data.""",
    # Tools for Supabase and Pinecone interaction will be assigned here
    verbose=True,
    allow_delegation=False, # Este agente se enfoca solo en la carga
    # Asigna el modelo Mistral cargado o su wrapper compatible aquí para el razonamiento del agente
    llm=mistral_llm_for_crewai # Usamos el wrapper compatible con CrewAI para el LLM
)

print("Agente de Carga definido.")

Agente de Carga definido.


In [ ]:
from crewai import Task
from typing import List, Dict, Any

# Asegúrate de que el Agente de Carga (loading_agent) ya esté definido.

class LoadingTasks:
    def __init__(self, agent):
        self.agent = agent

    def load_structured_data_to_supabase(self):
        return Task(
            description=(
                "Receive structured data (dictionary or list of dictionaries) extracted from the processing stage."
                "Identify the target Supabase table based on the data structure or provided metadata."
                "Use the appropriate tool to insert or update the data in the specified Supabase table."
                "Handle potential database errors or conflicts."
            ),
            expected_output=(
                "A report summarizing the loading operation for Supabase, including the number of records "
                "inserted/updated and any errors encountered."
            ),
            agent=self.agent,
            # Tool for Supabase interaction will be assigned here.
        )

    def load_vector_data_to_pinecone(self):
        return Task(
            description=(
                "Receive a list of vectors with their metadata, prepared for Pinecone insertion."
                "Identify the target Pinecone index based on the metadata (e.g., documentos-informes-vector, noticia-relevante-vector)."
                "Use the appropriate tool to upsert (insert/update) the vectors into the specified Pinecone index."
                "Handle batching for efficient loading and any API errors."
            ),
            expected_output=(
                "A report summarizing the loading operation for Pinecone, including the number of vectors "
                "upserted and any errors encountered."
            ),
            agent=self.agent,
            # Tool for Pinecone interaction will be assigned here.
        )

# Example of how to instantiate these tasks (requires loading_agent to be defined):
# loading_tasks = LoadingTasks(loading_agent)
# supabase_load_task = loading_tasks.load_structured_data_to_supabase()
# pinecone_load_task = loading_tasks.load_vector_data_to_pinecone()

print("Tareas para el Agente de Carga definidas.")

Tareas para el Agente de Carga definidas.


In [ ]:
from crewai.tools import tool
from typing import List, Dict, Any, Union
# Asegúrate de que las instancias de cliente de Supabase y Pinecone estén disponibles
# from supabase import create_client, Client
# from pinecone import Pinecone
# supabase: Client = ...
# pc: Pinecone = ...

@tool
def load_data_to_supabase(data: Union[Dict[str, Any], List[Dict[str, Any]]], table_name: str) -> str:
    """
    Loads structured data (dictionary or list of dictionaries) into a specified Supabase table.
    Handles both single records and lists of records.
    Performs insertion or update based on primary keys if the table schema supports it.
    Returns a summary of the operation result.
    """
    print(f"Using load_data_to_supabase tool for table: {table_name}")

    if not data:
        return f"No data provided to load into {table_name}."

    # Ensure data is a list for batch upsert, even if it's a single dictionary
    data_list = data if isinstance(data, list) else [data]

    try:
        # Use the Supabase client to upsert data
        # The `upsert` method handles inserting new records and updating existing ones
        # based on conflicts on primary keys or unique constraints.
        # Ensure your Supabase table has defined primary keys or unique constraints
        # for upsert to work correctly.
        response = supabase.from_(table_name).upsert(data_list).execute()

        # Check for errors in the response
        if response.data:
             inserted_or_updated_count = len(response.data)
             return f"Successfully loaded {inserted_or_updated_count} records into Supabase table '{table_name}'."
        elif response.error:
             print(f"Supabase loading error for table {table_name}: {response.error}")
             return f"Error loading data into Supabase table '{table_name}': {response.error.message}"
        else:
             # This case might occur if upsert did nothing (e.g., data was identical) or response structure is unexpected
             return f"Supabase upsert operation completed for table '{table_name}', but no data was returned in the response."


    except Exception as e:
        print(f"An unexpected error occurred during Supabase loading for table {table_name}: {e}")
        return f"An unexpected error occurred during Supabase loading for table '{table_name}': {e}"


@tool
def load_vectors_to_pinecone(vectors: List[Dict[str, Any]], index_name: str) -> str:
    """
    Loads a list of vectors with metadata into a specified Pinecone index.
    Each item in the list should be a dictionary with 'id', 'values', and 'metadata'.
    Performs upsert operation in Pinecone. Handles batching automatically or manually if needed.
    Returns a summary of the operation result.
    """
    print(f"Using load_vectors_to_pinecone tool for index: {index_name}")

    if not vectors:
        return f"No vectors provided to load into Pinecone index {index_name}."

    try:
        # Get the Pinecone index
        index = pc.Index(index_name)

        # Upsert vectors into the index
        # The upsert method can take a list of vectors
        # The Pinecone client handles batching internally by default
        upsert_response = index.upsert(vectors=vectors)

        # The response structure might vary, check Pinecone SDK docs for details
        # A successful upsert often returns an object with upserted_count
        if hasattr(upsert_response, 'upserted_count'):
             return f"Successfully upserted {upsert_response.upserted_count} vectors into Pinecone index '{index_name}'."
        else:
             # Fallback if response structure is different
             return f"Pinecone upsert operation completed for index '{index_name}'. Response: {upsert_response}"


    except Exception as e:
        print(f"An unexpected error occurred during Pinecone loading for index {index_name}: {e}")
        return f"An unexpected error occurred during Pinecone loading for index '{index_name}': {e}"

# These tools can now be assigned to the LoadingAgent
# loading_agent.tools = [load_data_to_supabase, load_vectors_to_pinecone]

print("Herramientas 'load_data_to_supabase' y 'load_vectors_to_pinecone' definidas.")

Herramientas 'load_data_to_supabase' y 'load_vectors_to_pinecone' definidas.


In [ ]:
from crewai import Crew, Process

# Asegúrate de que los agentes y las clases de tareas estén definidos
# (extraction_agent, structured_processing_agent, vectorial_processing_agent, loading_agent)
# (ExtractionTasks, StructuredProcessingTasks, VectorialProcessingTasks, LoadingTasks)

# Asegúrate de que las herramientas estén asignadas a los agentes donde sea necesario
# extraction_agent.tools = [extraction_tool]
# structured_processing_agent.tools = [extract_structured_data]
# vectorial_processing_agent.tools = [chunk_and_embed_content] # chunk_and_embed_content es una herramienta definida con @tool
# loading_agent.tools = [load_data_to_supabase, load_vectors_to_pinecone]

# Instantiate the task classes with their respective agents
extraction_tasks = ExtractionTasks(extraction_agent)
structured_tasks = StructuredProcessingTasks(structured_processing_agent)
vectorial_tasks = VectorialProcessingTasks(vectorial_processing_agent)
loading_tasks = LoadingTasks(loading_agent)

# Define the tasks and their dependencies/sequence
# The output of one task becomes the input of the next
extract_data_task = extraction_tasks.download_and_scrape_content()

# Note: The output of extract_data_task will be a list of raw data items.
# The structured and vectorial processing tasks will need to handle processing
# each item in this list, potentially in parallel or sequentially.
# For a simple sequential flow, the *next* task will receive the full list.
# A more complex Crew setup might involve breaking down the list and processing items individually.
# For this example, let's assume the next tasks are designed to iterate over the input list.

# Tasks for Structured Processing (these would likely iterate over the output of extract_data_task)
# We'll need to refine how the output of extraction maps to these tasks based on content_type
# For now, define the tasks themselves:
process_structured_financial = structured_tasks.process_financial_reports()
process_structured_daily_movements = structured_tasks.process_daily_movements_json()
process_structured_macroeconomic = structured_tasks.process_macroeconomic_data()
process_structured_public_contracts = structured_tasks.process_public_contracts()
process_structured_general_reports = structured_tasks.process_general_reports_metadata()


# Tasks for Vectorial Processing (these would likely iterate over raw content or processed text)
# The chunking task needs raw content (text or file path)
chunk_and_embed_task = vectorial_tasks.chunk_and_embed_content_task()
prepare_metadata_task = vectorial_tasks.prepare_pinecone_metadata_task() # This task needs chunks + source info


# Tasks for Loading (these need structured data or vectors with metadata)
load_supabase_task = loading_tasks.load_structured_data_to_supabase()
load_pinecone_task = loading_tasks.load_vector_data_to_pinecone()


# Define the overall pipeline flow using a sequential process
# This is a simplified representation. In a real scenario, you'd need to manage
# the flow based on content types and ensure the output of one task
# is correctly formatted as input for the next.
# CrewAI's sequential process feeds the output of task N to task N+1.
# This might require intermediate tasks or careful design of task inputs/outputs.

# A more realistic flow might involve:
# 1. Extraction Task (output: list of raw items)
# 2. Dispatcher Task (takes list, identifies item type, delegates to appropriate processing task)
# 3. Structured Processing Tasks (output: structured data)
# 4. Vectorial Processing Tasks (output: vectors with metadata)
# 5. Aggregator Task (collects results from processing tasks)
# 6. Loading Tasks (takes aggregated structured data and vectors, loads into DBs)

# For a basic sequential Crew example, let's chain some representative tasks.
# NOTE: This chaining assumes the output format is compatible between tasks,
# which needs careful implementation in the task execution logic.
# We'll need to adjust this significantly when building the full pipeline.

# Example simplified sequential flow:
# Extract -> Process Structured (e.g., Financial) -> Load Supabase
# Extract -> Process Vectorial (Chunk & Embed) -> Prepare Metadata -> Load Pinecone

# Let's create a basic Crew with the defined agents and some tasks
# We'll need to decide on the actual flow and task inputs/outputs when building the final Crew.

# For now, we can just list the agents and tasks we've defined.
# The actual Crew instantiation with the correct task graph will come next.

print("Agentes y Tareas definidas y listas para ser integradas en un Crew.")


Agentes y Tareas definidas y listas para ser integradas en un Crew.

Próximo paso: Definir el flujo exacto y crear la instancia del Crew.


In [ ]:
from crewai import Crew, Process

# Asegúrate de que los agentes y las tareas estén definidos y las herramientas asignadas
# (extraction_agent, structured_processing_agent, vectorial_processing_agent, loading_agent)
# (extraction_tasks, structured_tasks, vectorial_tasks, loading_tasks)
# Asegúrate de que las herramientas estén asignadas a los agentes donde sea necesario:
# extraction_agent.tools = [extraction_tool]
# structured_processing_agent.tools = [extract_structured_data]
# vectorial_processing_agent.tools = [chunk_and_embed_content]
# loading_agent.tools = [load_data_to_supabase, load_vectors_to_pinecone]

# Instantiate the task classes with their respective agents
# Note: Instantiating with agents here automatically assigns the agent to the tasks created by these instances
extraction_tasks = ExtractionTasks(extraction_agent)
structured_tasks = StructuredProcessingTasks(structured_processing_agent)
vectorial_tasks = VectorialProcessingTasks(vectorial_processing_agent)
loading_tasks = LoadingTasks(loading_agent)

# Define the tasks and their dependencies/sequence
# The output of one task becomes the input of the next

# Re-instantiate tasks and explicitly assign agents if not already done by the Task class init
# Our Task classes already assign the agent in __init__, so we just need to call the methods
extract_data_task_instance = extraction_tasks.download_and_scrape_content()
process_structured_task_instance = structured_tasks.process_general_reports_metadata() # Example structured task
chunk_and_embed_task_instance = vectorial_tasks.chunk_and_embed_content_task()
prepare_metadata_task_instance = vectorial_tasks.prepare_pinecone_metadata_task()
load_supabase_task_instance = loading_tasks.load_structured_data_to_supabase()
load_pinecone_task_instance = loading_tasks.load_vector_data_to_pinecone()


# Define the list of tasks for the Crew in sequential order
# This sequence is illustrative and needs to be adjusted for the actual data flow
# For a real pipeline, you'd need logic to decide which processing/loading tasks to run
# based on the output of the extraction step.
# A simple sequential list might look like:
# [extract_data_task_instance, some_processing_task, some_loading_task]

# Let's define a sample sequential task list assuming a flow where
# extraction leads to both structured and vectorial processing, and then loading.
# NOTE: This simple list doesn't handle the branching needed for different content types.
# A more advanced Crew setup or intermediate tasks would be required for that.

# For now, let's define a minimal sequential flow to create the Crew instance:
# Extract -> Chunk & Embed -> Prepare Metadata -> Load Pinecone (focus on vectorial path)
# We would need a separate path or a more complex graph for the structured path.

# Let's create a Crew with a simple sequential flow focusing on vector ingestion
# and assume a separate crew or logic for structured ingestion, or refine this later.

# A more realistic sequential flow for a document that needs both structured and vectorial processing might be:
# Extract -> Process Structured Metadata -> Chunk & Embed -> Prepare Vector Metadata -> Load Structured -> Load Vector
# This still requires tasks designed to pass the necessary information between stages.

# Let's create a Crew with a sequential process using the tasks we defined,
# acknowledging that the actual data passing and branching logic needs refinement.

# Example simplified sequential task list:
pipeline_tasks = [
    extract_data_task_instance,
    # How to handle the output of extraction (list of items) and feed to appropriate processing?
    # This is where the Crew's design gets complex.
    # For now, let's add the processing tasks assuming they can handle inputs correctly.
    # This is a placeholder sequence and needs refinement based on how tasks consume input.
    # Ensure correct tasks are chained based on expected output and input types
    # This sequence assumes the output of extract_data_task_instance can be processed
    # by process_structured_task_instance, etc. This needs careful design.
    process_structured_task_instance, # Example: process general report metadata from extracted data
    chunk_and_embed_task_instance, # Example: chunk and embed text content from extracted data or processed data
    prepare_metadata_task_instance, # Example: prepare metadata from chunks and (potentially) structured data
    load_supabase_task_instance, # Example: load structured data (output of process_structured_task_instance)
    load_pinecone_task_instance # Example: load vectors (output of prepare_metadata_task_instance)
]


# Instantiate the Crew
inverbot_pipeline_crew = Crew(
    agents=[
        extraction_agent,
        structured_processing_agent,
        vectorial_processing_agent,
        loading_agent
    ],
    tasks=pipeline_tasks,
    process=Process.sequential, # Define the process flow
    verbose=True # Set to True for detailed output during execution
)

print("\nInstancia del Crew 'inverbot_pipeline_crew' creada con un proceso secuencial.")
print("Nota: La definición exacta del flujo de tareas y la gestión de inputs/outputs entre tareas")
print("necesita ser validada y ajustada cuidadosamente para el pipeline completo.")


Instancia del Crew 'inverbot_pipeline_crew' creada con un proceso secuencial.
Nota: La definición exacta del flujo de tareas y la gestión de inputs/outputs entre tareas
necesita ser validada y ajustada cuidadosamente para el pipeline completo.


In [ ]:
# Definición de fuentes de datos de prueba para debugging
data_sources_test = [
    {
        "category": "Test Page",
        "url": "https://www.africau.edu/images/default/sample.pdf", # URL de un PDF de ejemplo
        "content_type": ["PDF"],
        "description": "Página de prueba con un enlace a un PDF simple."
    },
    {
        "category": "Test Page with Text",
        "url": "https://www.w3schools.com/html/html_paragraphs.asp", # Página con texto y HTML simple
        "content_type": ["TEXT"],
        "description": "Página de prueba con contenido de texto simple."
    }
    # Puedes añadir más fuentes de prueba aquí según necesites
    # {
    #     "category": "Test JSON Feed",
    #     "url": "URL_DE_UN_JSON_DE_PRUEBA",
    #     "content_type": ["JSON"],
    #     "description": "Feed JSON de prueba."
    # }
]

print("Variable 'data_sources_test' definida con fuentes de prueba.")
# Puedes imprimir la lista para verificar su contenido
# display(data_sources_test)

Variable 'data_sources_test' definida con fuentes de prueba.


In [ ]:
# Asegúrate de que la instancia del Crew 'inverbot_pipeline_crew' esté definida
# y de que 'data_sources_test' contenga los datos de prueba.

# Para ejecutar el Crew con los datos de prueba, le pasamos un diccionario
# al método kickoff(). La clave del diccionario debe coincidir con el
# nombre del parámetro de entrada esperado por la primera tarea en el pipeline.
# En este caso, la primera tarea es 'download_and_scrape_content'
# y espera la lista de fuentes como input.

# Nota: La tarea 'download_and_scrape_content' no tiene un parámetro explícito
# definido en su descripción como 'sources_list'. CrewAI secuencialmente
# pasa el OUTPUT de la tarea N al INPUT de la tarea N+1.
# Para la *primera* tarea, el input se pasa a través del diccionario en kickoff().
# El nombre de la clave en el diccionario de kickoff() se convierte en la entrada
# para la primera tarea. Podemos nombrarla intuitivamente, como 'sources'.

# Vamos a intentar ejecutar el Crew pasando 'data_sources_test' como input a la primera tarea.
# El agente de extracción deberá usar su herramienta 'Extraction Tool' que,
# a su vez, utiliza la lógica de ExtractionLogic.
# La herramienta 'Extraction Tool' actualmente no acepta un argumento 'sources'.
# Necesitamos ajustar la herramienta 'Extraction Tool' para que use el input de la tarea.

# Ajustemos la herramienta Extraction Tool para que acepte el input y lo pase a ExtractionLogic.
# La herramienta actual llama a extraction_handler_instance.run_extraction() sin argumentos,
# y esa instancia fue creada con data_sources originales.

# Modifiquemos la herramienta 'Extraction Tool' (cell 8c683727) para que reciba el input
# de la tarea y lo use con una nueva instancia de ExtractionLogic o ajuste la existente.

# Vamos a modificar la herramienta Extraction Tool para que reciba 'sources_list'
# como argumento y use esos datos para la extracción.

# Primero, modifico la herramienta (esto debería estar en una celda modify_cells)
# --- Inicio de la modificación conceptual de la herramienta ---
# @tool
# def extraction_tool_modified(sources_list: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
#     """
#     A tool to extract raw content from provided data sources.
#     Input is a list of dictionaries, each with 'category', 'url', and 'content_type'.
#     """
#     print(f"Extraction Tool received sources_list with {len(sources_list)} items.")
#     # Creamos una nueva instancia de ExtractionLogic con los datos de entrada
#     temp_extraction_handler = ExtractionLogic(data_sources=sources_list)
#     return temp_extraction_handler.run_extraction()
# --- Fin de la modificación conceptual de la herramienta ---

# Ahora, asumimos que la herramienta ha sido modificada para aceptar 'sources_list'
# y que el extraction_agent usa esta versión modificada de la herramienta.

# Re-asignamos la herramienta modificada al agente de extracción (esto debería estar en una celda modify_cells)
# extraction_agent.tools = [extraction_tool_modified]

# Y ahora, ejecutamos el Crew pasando data_sources_test como input a la primera tarea.
# La clave del diccionario en kickoff() debe coincidir con el nombre del parámetro
# que la primera tarea (y por lo tanto, la herramienta que usa) espera.
# Si la herramienta espera 'sources_list', usamos 'sources_list' como clave.

# Intentaremos ejecutar el Crew con el input 'sources_list'.
# Si la herramienta no está ajustada para recibir este input, fallará.
# Debemos asegurar que la herramienta que usa la primera tarea acepte este input.

# La herramienta 'Extraction Tool' (cell 8c683727) necesita ser modificada para aceptar un argumento.
# La descripción de la tarea 'download_and_scrape_content' (cell ac459931)
# menciona que el input "should be a list of dictionaries...".
# CrewAI intentará pasar el diccionario de kickoff() a la primera tarea.
# La clave del diccionario se convierte en el único argumento si la tarea/herramienta
# espera un solo argumento posicional, o en argumentos nombrados si la herramienta
# espera argumentos con nombres coincidentes.

# Vamos a asumir que la herramienta 'Extraction Tool' espera un argumento posicional
# que es la lista de fuentes.
# Modifico la herramienta 'Extraction Tool' (cell 8c683727) para aceptar un argumento.
# --- Inicio de la modificación de la herramienta 'Extraction Tool' ---
# @tool
# def extraction_tool(sources_list: List[Dict[str, Any]]) -> List[Dict[str, Any]]: # Aceptar sources_list como argumento
#     """
#     A tool to extract raw content from provided data sources.
#     Input is a list of dictionaries, each with 'category', 'url', and 'content_type'.
#     """
#     print(f"Extraction Tool received sources_list with {len(sources_list)} items.")
#     # Creamos una nueva instancia de ExtractionLogic con los datos de entrada
#     temp_extraction_handler = ExtractionLogic(data_sources=sources_list)
#     return temp_extraction_handler.run_extraction()
# --- Fin de la modificación de la herramienta 'Extraction Tool' ---

# Ahora, modifico la celda 8c683727 para reflejar esto. Luego, ejecutamos esta celda.
# Después de ejecutar la celda 8c683727 modificada, podremos ejecutar la celda actual.

print("Preparando para ejecutar el Crew con data_sources_test.")
print("Nota: La herramienta 'Extraction Tool' (cell 8c683727) debe ser modificada y ejecutada primero para aceptar el input 'sources_list'.")

# Una vez que la herramienta esté modificada y la celda 8c683727 ejecutada,
# descomenta y ejecuta el siguiente bloque para iniciar el Crew:

try:
    print("\nIniciando la ejecución del Crew con data_sources_test...")
    # La clave 'sources_list' aquí debe coincidir con el nombre del argumento
    # que la herramienta 'Extraction Tool' espera.
    result = inverbot_pipeline_crew.kickoff(inputs={'sources_list': data_sources_test})
    print("\n--- Ejecución del Crew Finalizada ---")
    print(result)
except Exception as e:
    print(f"\nError durante la ejecución del Crew: {e}")

Preparando para ejecutar el Crew con data_sources_test.
Nota: La herramienta 'Extraction Tool' (cell 8c683727) debe ser modificada y ejecutada primero para aceptar el input 'sources_list'.

Iniciando la ejecución del Crew con data_sources_test...


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: a22cb321-c587-4ec6-8cdd-a96462a020cf                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Data Extractor                                                                                          │
│                                                                                                                 │
│  Task: Iterate through the provided data sources, download or scrape content based on type.                     │
│  Handle different content types: JSON (fetch data), TEXT (scrape HTML), PDF/EXCEL/PNG (download file).          │
│  Return a list of dictionaries, each containing source info and raw content/file path.                          │
│  Input should be a list of dictionaries, each with 'category', 'url', and 'content_type'.                       │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭─────────────────────────────────────────────────── LLM Error ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  ❌ LLM Call Failed                                                                                             │
│  Error: litellm.BadRequestError: HuggingfaceException - {"error":{"message":"The requested model                │
│  'mistralai/Mistral-7B-Instruct-v0.3' is not supported by any provider you have                                 │
│  enabled.","type":"invalid_request_error","param":"model","code":"model_not_supported"}}                        │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────── Task Failure ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Failed                                                                                                    │
│  Name: c1618bf8-818d-48b2-9cd0-4977c66e0f16                                                                     │
│  Agent: Data Extractor                                                                                          │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────── Crew Failure ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Failed                                                                                          │
│  Name: crew                                                                                                     │
│  ID: a22cb321-c587-4ec6-8cdd-a96462a020cf                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output:                                                                                                  │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯


Error durante la ejecución del Crew: litellm.BadRequestError: HuggingfaceException - {"error":{"message":"The requested model 'mistralai/Mistral-7B-Instruct-v0.3' is not supported by any provider you have enabled.","type":"invalid_request_error","param":"model","code":"model_not_supported"}}
